In [1]:
import statsmodels.api as sm
import pylab as pl
import numpy as np
import MySQLdb as mdb
import getpass

In [2]:
# Connect to the MySQL database containing the results of import_game_pbp_data.py
rootpass = getpass.getpass('Enter the database root password: ')
con = mdb.connect('localhost', 'root', rootpass, 'NFL_Offensive_Plays_2015')
cur = con.cursor()

Enter the database root password: ········


In [79]:
# This cell queries the database and performs logistic regression on the offensive play data for each team. The regression model
# is trained on the first 8 weeks of data, and tries to predict runs or passes for the remaining 9 weeks of data.

# A list of the teams. Teams that switched offensive coordinators during the 2015 season: Detroit Lions (Oct 26, after Week 7), 
# Indianapolis Colts (Nov 3, after Week 8), St. Louis Rams (Dec. 7, after Week 13)
teams = ['Baltimore Ravens', 'Cincinnati Bengals', 'Cleveland Browns', 'Pittsburgh Steelers', 'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars', 'Tennessee Titans', 'Buffalo Bills', 'Miami Dolphins', 'New England Patriots', 'New York Jets', 'Denver Broncos', 'Kansas City Chiefs', 'Oakland Raiders', 'San Diego Chargers', 'Chicago Bears', 'Detroit Lions', 'Green Bay Packers', 'Minnesota Vikings', 'Atlanta Falcons', 'Carolina Panthers', 'New Orleans Saints', 'Tampa Bay Buccaneers', 'Dallas Cowboys', 'New York Giants', 'Philadelphia Eagles', 'Washington Redskins', 'Arizona Cardinals', 'St. Louis Rams', 'San Francisco 49ers', 'Seattle Seahawks']
#Below, I initialize the lists of parameters I'll use for later analysis.
logitList = []
fitResult = []
predIsTrue = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
fracCorrect = []
numPass = []
fracPass = []
fracTruePass = []
fracFalsePass = []
fracTrueRun = []
fracFalseRun = []
fracUncertain = []
fracUncertainOrSureRun = []
F1Pass = []
F1Run = []
# A list of the total yards (and points) for each team, in the order corresponding to that of teams.
totYds = [5749,5728,5311,6327,5564,5142,5581,4988,5775,5307,5991,5925,5688,5299,5336,5949,5514,5547,5353,5139,5990,5871,6461,6014,5361,5956,5830,5661,6533,4761,4860,6058]
pts = [328,419,278,423,339,333,376,299,379,310,465,387,355,405,359,320,335,358,368,365,339,500,408,342,275,420,377,388,489,280,238,423]

# For each team, I will perform logistic regression on the offensive plays data from the first half of the season to try to 
# predict whether plays from the second half of the season will be a run or a pass.
j = 0
while j < len(teams):
    #The first step is to get the data from the first half of the season from the MySQL database.
    cur.execute("SELECT `Time Remaining`, Down, `To Go`, `Field Position`, `Score Differential` FROM `" + teams[j] + "` WHERE Week < 9")
    firstHalfData = cur.fetchall()
    cur.execute("SELECT IsPass FROM `" + teams[j] + "` WHERE Week < 9")
    firstHalfOut = cur.fetchall()
    
    # I call the inputs from the training set  firstHalfIn. Each row in firstHalfIn includes the bias term, the down,
    # the distance, the field position (a number between 0 and 50; it doesn't differentiate between sides of the field), and 
    # (score differential) divided by (time remaining in seconds plus .01). I expect the probability of a pass to change 
    # monotonically with each non-bias input: The higher the down, the higher the distance, the farther from either goalline, and
    # the lower the ratio between score differential and time remaining, the higher I expect the probability of a pass to be. The 
    # .01 is in the denominator of the last input to avoid singularities on the rare occasions that a defensive penalty allows for
    # an offensive play with no time remaining on the clock.
    firstHalfIn = []
    for play in firstHalfData:
        firstHalfIn.append([1, int(play[1]), int(play[2]), int(min(play[3], 100-play[3])), float(play[4])/(float(play[0])+.01)])

    # I save the results of the fit result in a list.
    logitList.append(sm.Logit(firstHalfOut, firstHalfIn))
    fitResult.append(logitList[j].fit())

    #Now, I get the data from the second half of the season (weeks 9-17)
    cur.execute("SELECT `Time Remaining`, Down, `To Go`, `Field Position`, `Score Differential` FROM `" + teams[j] + "` WHERE Week > 8")
    secondHalfData = cur.fetchall()
    cur.execute("SELECT IsPass FROM `" + teams[j] + "` WHERE Week > 8")
    secondHalfOut = cur.fetchall()
    secondHalfIn = []
    for play in secondHalfData:
        secondHalfIn.append([1, int(play[1]), int(play[2]), int(min(play[3], 100-play[3])), float(play[4])/(float(play[0])+.01)])

    # I now need to use the results of the logistic regression to make predictions about the second half of the season. Dotting
    # the second half data into the fitting parameters gives a list of sigmoid funtion arguments. Applying the sigmoid function 
    # to them gives a list of pass probabilites. Rounding these probabilities gives the prediction of a run or pass.
    secondHalfPred = []
    sigmoids = 1/(1+np.exp(-np.dot(secondHalfIn, fitResult[j].params)))
    for sigmoid in sigmoids:
        secondHalfPred.append(round(sigmoid))

    # The following while loop fills in the list predIsTrue[j], with ones indicating correct predictions, and zeros indicating
    # incorrect predictions. It also counts the number of true pass, true run, false pass, and false run predictions there are.
    # It also counts plays where the model is uncertain--defined as plays where the probability of a pass is between 40% and 
    # 50%--and plays where the probability of a pass is less than 60%. This allows for more analysis later on.
    
    i = 0
    tempTruePass = 0
    tempTrueRun = 0
    tempFalsePass = 0
    tempFalseRun = 0
    tempUncertain = 0
    tempUncertainOrSureRun = 0
    while i < len(secondHalfPred):
        if sigmoids[i] < .6:
            tempUncertainOrSureRun += 1
            if sigmoids[i] > .4:
                tempUncertain += 1
        if secondHalfPred[i] == 1 and secondHalfOut[i][0] == 1:
            predIsTrue[j].append(1)
            tempTruePass += 1
        elif secondHalfPred[i] == 0 and secondHalfOut[i][0] == 0:
            predIsTrue[j].append(1)
            tempTrueRun += 1
        elif secondHalfPred[i] == 1 and secondHalfOut[i][0] == 0:
            predIsTrue[j].append(0)
            tempFalsePass += 1
        elif secondHalfPred[i] == 0 and secondHalfOut[i][0] == 1:
            predIsTrue[j].append(0)
            tempFalseRun += 1
        i += 1
    
    # We now calculate the fraction of second half plays that were correct predictions, correct pass predictions, correct run 
    # predictions, false pass predictions, false run predictions, uncertain predictions, and predictions that are uncertain or 
    # sure of a run.
    fracCorrect.append(float(sum(predIsTrue[j]))/float(len(predIsTrue[j])))
    fracTruePass.append(float(tempTruePass)/float(len(predIsTrue[j])))
    fracTrueRun.append(float(tempTrueRun)/float(len(predIsTrue[j])))
    fracFalsePass.append(float(tempFalsePass)/float(len(predIsTrue[j])))
    fracFalseRun.append(float(tempFalseRun)/float(len(predIsTrue[j])))
    fracUncertain.append(float(tempUncertain)/float(len(predIsTrue[j])))
    fracUncertainOrSureRun.append(float(tempUncertainOrSureRun)/float(len(predIsTrue[j])))
    
    # The next few lines find the total number of passing plays, as well as the fraction of pass plays in the second half of the
    # season.
    tempNumPass = 0
    for play in secondHalfOut:
        tempNumPass += play[0]
    numPass.append(tempNumPass)
    fracPass.append(float(numPass[j])/float(len(secondHalfOut)))
    
    j += 1

Optimization terminated successfully.
         Current function value: 0.591050
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.654667
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.573735
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.611577
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.575147
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.582194
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.606127
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.599642
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.601416
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.601603
 

In [44]:
# A list of the fraction of plays the model got correct for each team.
fracCorrect

[0.6789667896678967,
 0.68,
 0.6535433070866141,
 0.55893536121673,
 0.6311787072243346,
 0.6360078277886497,
 0.6703296703296703,
 0.6610486891385767,
 0.6648550724637681,
 0.6338797814207651,
 0.6303972366148531,
 0.6766256590509666,
 0.6454388984509466,
 0.5803571428571429,
 0.6289752650176679,
 0.6679389312977099,
 0.6696588868940754,
 0.6257309941520468,
 0.5953125,
 0.7102803738317757,
 0.6750972762645915,
 0.6473594548551959,
 0.6844106463878327,
 0.6433566433566433,
 0.6819047619047619,
 0.6383763837638377,
 0.5958132045088567,
 0.5959780621572212,
 0.6400778210116731,
 0.7038461538461539,
 0.658008658008658,
 0.6227897838899804]

In [8]:
# The fraction of plays that were passes for each team.
fracPass

[0.6568265682656826,
 0.5290909090909091,
 0.6259842519685039,
 0.6615969581749049,
 0.5437262357414449,
 0.6203522504892368,
 0.6703296703296703,
 0.5936329588014981,
 0.4927536231884058,
 0.6284153005464481,
 0.6044905008635578,
 0.6028119507908611,
 0.5972461273666093,
 0.5200892857142857,
 0.6183745583038869,
 0.6297709923664122,
 0.5224416517055656,
 0.6101364522417154,
 0.6140625,
 0.5158878504672897,
 0.6186770428015564,
 0.4991482112436116,
 0.6026615969581749,
 0.5804195804195804,
 0.5866666666666667,
 0.5940959409594095,
 0.5668276972624798,
 0.5850091407678245,
 0.5953307392996109,
 0.551923076923077,
 0.6385281385281385,
 0.49901768172888017]

In [63]:
# The fraction of plays where the model was uncertain for each team. There is a surprising amount of variance here.
fracUncertain

[0.48523985239852396,
 0.649090909090909,
 0.4015748031496063,
 0.3669201520912547,
 0.44106463878326996,
 0.48140900195694714,
 0.47802197802197804,
 0.41198501872659177,
 0.4963768115942029,
 0.21311475409836064,
 0.10535405872193437,
 0.4182776801405975,
 0.3270223752151463,
 0.5513392857142857,
 0.3568904593639576,
 0.4312977099236641,
 0.3087971274685817,
 0.4502923976608187,
 0.5078125,
 0.5626168224299065,
 0.3949416342412451,
 0.3577512776831346,
 0.4296577946768061,
 0.27972027972027974,
 0.3980952380952381,
 0.3929889298892989,
 0.5442834138486312,
 0.4954296160877514,
 0.37937743190661477,
 0.24807692307692308,
 0.4588744588744589,
 0.5913555992141454]

In [127]:
# Here, I construct a list called difference. Each element of difference is equal to the fraction of plays the model got correct
# and the fraction of passing plays.
difference = []
i = 0
while i < len(fracCorrect):
    difference.append(fracCorrect[i] - fracPass[i])
    i += 1
difference

[0.022140221402214055,
 0.150909090909091,
 0.027559055118110187,
 -0.1026615969581749,
 0.0874524714828897,
 0.015655577299412915,
 0.0,
 0.0674157303370786,
 0.17210144927536225,
 0.005464480874316946,
 0.025906735751295318,
 0.07381370826010547,
 0.04819277108433728,
 0.060267857142857206,
 0.010600706713780994,
 0.03816793893129766,
 0.14721723518850982,
 0.015594541910331383,
 -0.018749999999999933,
 0.19439252336448598,
 0.05642023346303504,
 0.14821124361158428,
 0.08174904942965777,
 0.06293706293706292,
 0.09523809523809523,
 0.04428044280442811,
 0.02898550724637683,
 0.010968921389396646,
 0.04474708171206221,
 0.15192307692307694,
 0.019480519480519543,
 0.12377210216110024]

In [128]:
# Here, I find out for which teams fracCorrect - fracPass is nonpositive. These cases are significant, because for these teams 
# it is more accurate to guess pass on every play than to use this model. There are three teams for which this is true: the 
# Pittsburgh Steelers, the Jacksonville Jaguars, and the Green Bay Packers. Surprisingly, none of these teams switched offensive 
# coordinators mid-season, which is what I would have expected. For the Steelers, it makes sense: due to a mixture of suspensions
# and injuries, at different points of the season they did not have access to their #1 runningback, wide receiver, and 
# quarterback. This means that playcalling varied from game to game much more than it did for most teams. The Jaguars also had
# injuries to several wide receivers and running backs, which perhaps affected their playcalling throughout the season as well.
# The same is not true for the Packers, although their best wide receiver was out the whole season due to an injury during the 
# preseason. Their offense did struggle early in the season, but largely recovered by the end of the season. Perhaps their 
# play calls early on were different from those later.
i = 0
while i < len(difference):
    if difference[i] <= 0:
        print teams[i]
    i += 1

Pittsburgh Steelers
Jacksonville Jaguars
Green Bay Packers


In [129]:
# Here, I find out for which teams our model does at least 10 percentage points better than guessing pass on every play. I 
# expected the teams on this list to have poor offenses during 2015, since it indicates that they were predictable. Some of them 
# were dismal; However, this list also includes the Carolina Panthers, who had arguably the best offense that season. Perhaps 
# the Panthers' players were able to execute particularly well, overcoming the predictability.
i = 0
while i < len(difference):
    if difference[i] >= .1:
        print teams[i]
    i += 1

Cincinnati Bengals
Buffalo Bills
Chicago Bears
Minnesota Vikings
Carolina Panthers
St. Louis Rams
Seattle Seahawks


In [53]:
# In this cell and those below, I try to find correlations between different measures of predictability and offensive success.
# They are often correlated in the way one would expect (predictabile offenses usually did worse), but none of the correlations
# were statistically significant with a 95% confidence interval. Perhaps, if I included data from multiple seasons, I could find
# statistically significant results.

linfitCorrect = sm.OLS(totYds, sm.add_constant(fracCorrect))
linfitCorrectresult = linfitCorrect.fit()
xvalsCorrect = [.55, .72]
yvalsCorrect = [linfitCorrectresult.params[1]*xvalsCorrect[0] + linfitCorrectresult.params[0], linfitCorrectresult.params[1]*xvalsCorrect[1] + linfitCorrectresult.params[0]]
plot1 = pl.plot(fracCorrect,totYds,'gx', xvalsCorrect, yvalsCorrect)
pl.ylabel('Total Yards')
pl.xlabel('Fraction of Correct Predictions')

In [54]:
pl.show(plot1)

In [60]:
linfitdiff = sm.OLS(totYds, sm.add_constant(difference))
linfitdiffresult = linfitdiff.fit()
xvalsdiff = [-.04, .14]
yvalsdiff = [linfitdiffresult.params[1]*xvalsdiff[0] + linfitdiffresult.params[0], linfitdiffresult.params[1]*xvalsdiff[1] + linfitdiffresult.params[0]]
plotdiff = pl.plot(difference, totYds, 'gx', xvalsdiff, yvalsdiff)

In [61]:
pl.show(plotdiff)

In [73]:
OLSUncertain = sm.OLS(pts, sm.add_constant(fracUncertain))
fitUncertain = OLSUncertain.fit()
xvalsUncertain = [.1, .7]
yvalsUncertain = [fitUncertain.params[1]*xvalsUncertain[0] + fitUncertain.params[0], fitUncertain.params[1]*xvalsUncertain[1] + fitUncertain.params[0]]
plotUncertain = pl.plot(fracUncertain, pts, 'gx', xvalsUncertain, yvalsUncertain)

In [74]:
pl.show(plotUncertain)

In [75]:
fitUncertain.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                   0.09067
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.765
Time:                        21:37:00   Log-Likelihood:                -176.35
No. Observations:                  32   AIC:                             356.7
Df Residuals:                      30   BIC:                             359.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        352.5868     42.647      8.267      0.000       265.489   439.685
x1            29.6101     98.334      0.301      0.765      -171.215   230.435
==============================================================================
Omnibus:                        0.879   Durbin-Watson:                   2.326
Prob(Omnibus):                  0.644   Jarque-Bera (JB):                0.446
Skew:                           0.289   Prob(JB):                        0.800
Kurtosis:                       3.028   Cond. No.                         10.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
OLSUncertainOrSureRun = sm.OLS(pts, sm.add_constant(fracUncertainOrSureRun))
fitUncertainOrSureRun = OLSUncertainOrSureRun.fit()
xvalsUncertainOrSureRun = [.1, .9]
yvalsUncertainOrSureRun = [fitUncertainOrSureRun.params[1]*xvalsUncertainOrSureRun[0] + fitUncertainOrSureRun.params[0], fitUncertainOrSureRun.params[1]*xvalsUncertainOrSureRun[1] + fitUncertainOrSureRun.params[0]]
plotUncertainOrSureRun = pl.plot(fracUncertainOrSureRun, pts, 'gx', xvalsUncertainOrSureRun, yvalsUncertainOrSureRun)

In [87]:
pl.show(plotUncertainOrSureRun)

In [85]:
fitUncertainOrSureRun.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.9173
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.346
Time:                        21:43:02   Log-Likelihood:                -175.91
No. Observations:                  32   AIC:                             355.8
Df Residuals:                      30   BIC:                             358.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        323.2772     44.878      7.203      0.000       231.623   414.931
x1            76.8888     80.282      0.958      0.346       -87.069   240.847
==============================================================================
Omnibus:                        0.833   Durbin-Watson:                   2.198
Prob(Omnibus):                  0.659   Jarque-Bera (JB):                0.376
Skew:                           0.264   Prob(JB):                        0.829
Kurtosis:                       3.061   Cond. No.                         9.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [123]:
OLSUncertainOrSureRunYds = sm.OLS(totYds, sm.add_constant(fracUncertainOrSureRun))
fitUncertainOrSureRunYds = OLSUncertainOrSureRunYds.fit()
xvalsUncertainOrSureRunYds = [.1, .9]
yvalsUncertainOrSureRunYds = [fitUncertainOrSureRunYds.params[1]*xvalsUncertainOrSureRunYds[0] + fitUncertainOrSureRunYds.params[0], fitUncertainOrSureRunYds.params[1]*xvalsUncertainOrSureRunYds[1] + fitUncertainOrSureRunYds.params[0]]
plotUncertainOrSureRunYds = pl.plot(fracUncertainOrSureRun, totYds, 'gx', xvalsUncertainOrSureRunYds, yvalsUncertainOrSureRunYds)

In [124]:
pl.show(plotUncertainOrSureRunYds)

In [120]:
fitUncertainOrSureRunYds.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.3894
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.537
Time:                        22:35:33   Log-Likelihood:                -239.02
No. Observations:                  32   AIC:                             482.0
Df Residuals:                      30   BIC:                             485.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5447.4263    322.511     16.891      0.000      4788.771  6106.082
x1           360.0099    576.936      0.624      0.537      -818.250  1538.270
==============================================================================
Omnibus:                        0.011   Durbin-Watson:                   2.061
Prob(Omnibus):                  0.994   Jarque-Bera (JB):                0.176
Skew:                          -0.031   Prob(JB):                        0.916
Kurtosis:                       2.642   Cond. No.                         9.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""